In [ ]:
file=r'/Users/lalitsachan/Dropbox/0.0 Data/census_income.csv'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
cd=pd.read_csv(file)

In [ ]:
cd.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
cd1,cd2=train_test_split(cd,test_size=0.2,random_state=0)

In [ ]:
# cd1.select_dtypes(include=['object']).columns
num_cols=['age', 'fnlwgt', 'education.num', 'capital.gain', 
          'capital.loss',
       'hours.per.week']
cat_cols=['workclass',  'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'native.country']

In [ ]:
# cd1.select_dtypes(include=['object']).columns

In [ ]:
class VarSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self,var_names):
        self.vars=var_names
    
    def fit(self,x,y=None):
        return self
    
    def transform(self,X):
        return X[self.vars]

In [ ]:
class get_dummies_Pipe(BaseEstimator, TransformerMixin):
    
    def __init__(self,freq_cutoff=0):
        self.freq_cutoff=freq_cutoff
        self.var_cat_dict={}
        
    def fit(self,x,y=None):
        
        data_cols=x.columns
        
        for col in data_cols:
            
            k=x[col].value_counts()
            cats=k.index[k>self.freq_cutoff][:-1]
            self.var_cat_dict[col]=cats
            
        return self
            
    def transform(self,x,y=None):
        
        dummy_data=x.copy()
        
        for col in self.var_cat_dict.keys():
            
            for cat in self.var_cat_dict[col]:
                name=col+'_'+cat
                dummy_data[name]=(dummy_data[col]==cat).astype(int)
            del dummy_data[col]
            
        return dummy_data

In [ ]:
num_pipe=Pipeline([
    ('num_var_select',VarSelector(num_cols)),
    ('standard',StandardScaler())
])

In [ ]:
cat_pipe=Pipeline([
    ('cat_var_select',VarSelector(cat_cols)),
    ('create_dummies',get_dummies_Pipe(100))
])

In [ ]:
data_pipe=Pipeline([
    ('features',FeatureUnion([
        ('cat_pipe',cat_pipe),
        ('num_pipe',num_pipe)
    ]))
])

prediction_pipeline=Pipeline([
    ('data_processing',data_pipe),
    ('clf',RandomforestClassifier(.....))
])

In [ ]:
data_pipe.fit(cd1)

In [ ]:
cd1=data_pipe.transform(cd1)

In [ ]:
cd1.shape

In [ ]:
cd2=data_pipe.transform(cd2)

In [ ]:
cd2.shape

In [ ]:
from sklearn.externals import joblib
joblib.dump(data_pipe,'data_pipeline.pkl')

## pipelines can also include predictive models 
## stacked models can also be converted into pipelines 
## pipelines can be tuned for hyperparameters

### using this data_pipeline

In [1]:
file=r'/Users/lalitsachan/Dropbox/0.0 Data/census_income.csv'
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.externals import joblib

from mypipes import *

In [2]:
data_pipe=open('data_pipeline.pkl','rb')

In [3]:
dp=joblib.load(data_pipe)

In [4]:
data=pd.read_csv(file)

In [5]:
ready_data=dp.transform(data)

/Users/lalitsachan/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [6]:
ready_data.shape

(32561, 43)